In [1]:
import pandas as pd
import numpy as np

In [2]:
filename = '../../Sample_Data/Processed/budget.title.basics.tsv'
df = pd.read_csv(filename, sep='\t')
df.head()

,tconst,name,date,prod_budget,domestic_gross,worldwide_gross,isAdult,startYear,runtimeMinutes,genres
0,tt4154796,Avengers: Endgame,2019,400000000,858373000,2797800564,0,2019,181,"Action,Adventure,Drama"
1,tt1298650,Pirates of the Caribbean: On Stranger Tides,2011,379000000,241063875,1045663875,0,2011,136,"Action,Adventure,Fantasy"
2,tt2395427,Avengers: Age of Ultron,2015,365000000,459005868,1403013963,0,2015,141,"Action,Adventure,Sci-Fi"
3,tt4154756,Avengers: Infinity War,2018,300000000,678815482,2048359754,0,2018,149,"Action,Adventure,Sci-Fi"
4,tt0974015,Justice League,2017,300000000,229024295,655945209,0,2017,120,"Action,Adventure,Fantasy"


In [3]:
n_rows = len(df)
df = df.drop(df[df['genres'] == '\\N'].index)
print('Removed {} rows'.format(n_rows-len(df)))

Removed 81 rows


In [4]:
genres_lists = df.genres.values.tolist()
genres_lists = [genres.split(',') for genres in genres_lists]

In [5]:
v = genres_lists
l = [len(x) for x in v]
f, u = pd.factorize(np.concatenate(v))
n, m = len(v), u.size
i = np.arange(n).repeat(l)

dummies = pd.DataFrame(
    np.bincount(i * m + f, minlength=n * m).reshape(n, m),
    df.index, u
)

df = df.drop('genres', 1).join(dummies)

In [6]:
df.head()

,tconst,name,date,prod_budget,domestic_gross,worldwide_gross,isAdult,startYear,runtimeMinutes,Action,...,Musical,Documentary,Adult,War,Biography,Western,Sport,Music,News,Film-Noir
0,tt4154796,Avengers: Endgame,2019,400000000,858373000,2797800564,0,2019,181,1,...,0,0,0,0,0,0,0,0,0,0
1,tt1298650,Pirates of the Caribbean: On Stranger Tides,2011,379000000,241063875,1045663875,0,2011,136,1,...,0,0,0,0,0,0,0,0,0,0
2,tt2395427,Avengers: Age of Ultron,2015,365000000,459005868,1403013963,0,2015,141,1,...,0,0,0,0,0,0,0,0,0,0
3,tt4154756,Avengers: Infinity War,2018,300000000,678815482,2048359754,0,2018,149,1,...,0,0,0,0,0,0,0,0,0,0
4,tt0974015,Justice League,2017,300000000,229024295,655945209,0,2017,120,1,...,0,0,0,0,0,0,0,0,0,0


In [9]:
for i in df.columns:
    print(df[i].describe())
    print('')
    if df[i].sum() == 0:
        print('all rows in {} is 0'.format(i))
        df.drop(i)

count          4705
unique         4622
top       tt8539810
freq              3
Name: tconst, dtype: object

count          4705
unique         4622
top       King Kong
freq              3
Name: name, dtype: object

count     4705
unique      92
top       2006
freq       216
Name: date, dtype: object

count    4.705000e+03
mean     3.452090e+07
std      4.278731e+07
min      5.000000e+03
25%      7.000000e+06
50%      2.000000e+07
75%      4.500000e+07
max      4.000000e+08
Name: prod_budget, dtype: float64

count    4.705000e+03
mean     4.672767e+07
std      6.998779e+07
min      3.880000e+02
25%      5.000000e+06
50%      2.292190e+07
75%      5.806019e+07
max      8.583730e+08
Name: domestic_gross, dtype: float64

count    4.705000e+03
mean     1.014424e+08
std      1.861662e+08
min      4.010000e+02
25%      8.300821e+06
50%      3.600000e+07
75%      1.103871e+08
max      2.797801e+09
Name: worldwide_gross, dtype: float64

count    4705.000000
mean        0.003613
std         0.0

In [ ]:
df.to_csv('../../Sample_Data/Processed/budget.genresplitted.tsv', sep='\t', index=False)